In [7]:
!pip install keras

In [11]:
!pip install tensorflow

In [22]:
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense
import numpy as np
# from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
# from keras.utils import np_utils
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as np_utils


In [16]:
corpus = pd.read_csv('processed_reviews.csv')

In [17]:
labels = corpus['label'].values

In [19]:
X = corpus['Cleaned']

# Fill NaN values with an empty string in the 'Cleaned' column
X.fillna('', inplace=True)

# Tokenize the text
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Pad sequences
X = pad_sequences(X, maxlen=100)



In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42) 

In [23]:
y_train_cat = np_utils.to_categorical(y_train, 2)
y_test_cat = np_utils.to_categorical(y_test, 2)


In [24]:
from tensorflow.keras import layers

In [25]:
rnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=20000, output_dim=32, input_shape=(X_train.shape[1],)),
    tf.keras.layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(layers.LSTM(128)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

2024-01-26 13:35:30.723800: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-26 13:35:30.724328: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [26]:
rnn_model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [27]:
rnn_model.fit(X_train, y_train_cat, epochs=3,
              validation_data=(X_test, y_test_cat),
              validation_steps=30)

Epoch 1/3
54/54 [==============================] - 14s 181ms/step - loss: 0.5491 - accuracy: 0.8396 - val_loss: 0.3740 - val_accuracy: 0.8788
Epoch 2/3
54/54 [==============================] - 8s 143ms/step - loss: 0.4013 - accuracy: 0.8635
Epoch 3/3
54/54 [==============================] - 8s 140ms/step - loss: 0.3993 - accuracy: 0.8635


In [28]:
y_test_pred = rnn_model.predict(X_test)
y_test_predclass = np.argmax(y_test_pred, axis=1)
y_trian_pred = rnn_model.predict(X_train)
y_train_predclass = np.argmax(y_trian_pred, axis=1)

54/54 [==============================] - 3s 47ms/step


In [29]:


from sklearn.metrics import accuracy_score,classification_report
print ("nDeep Neural Network - Test accuracy:",(round(accuracy_score(y_test, y_test_predclass),4)*100))
print ("nDeep Neural Network - Train accuracy:",(round(accuracy_score(y_train, y_train_predclass),4)*100))



nDeep Neural Network - Test accuracy: 77.16
nDeep Neural Network - Train accuracy: 75.2


In [33]:


X_pred=tokenizer.texts_to_sequences(['شهرکرد جای خیلی خوبی است.'])
X_pred=pad_sequences(X_pred,maxlen=100)



In [34]:
rnn_model.predict(X_pred)

1/1 [==============================] - 0s 26ms/step


array([[0.12946135, 0.87053865]], dtype=float32)